In [3]:
batch_size = 2
seed = 40
train_ratio = 0.75
test_ratio = 0.25
pred_per_image = 15
num_workers = 3

In [16]:
import torch
# Check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
# Check if GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")
save_directory = "/home/azureuser/cloudfiles/code/Users/shahriarizadehfatima"
import os
from PIL import Image, ImageDraw, ImageFont, ImageOps
import numpy as np
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch.nn.functional as F
from torchvision.transforms import functional as F_transform
from torchvision.transforms import Normalize as NormalizeTransform
from torchvision.ops.boxes import box_iou
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from albumentations import Compose, Resize, HorizontalFlip, BboxParams, VerticalFlip, Normalize
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import random
from azureml.core import Dataset, Datastore, Workspace
from torchmetrics.detection import MeanAveragePrecision

Using device: cuda


/anaconda/envs/Object_Detection/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [17]:
json_dir = "/home/azureuser/cloudfiles/code/Users/shahriarizadehfatima/config.json"
# Define folders to exclude
exclude_folders = {}
# Load workspace from config.json
ws = Workspace.from_config(json_dir)
datastore = Datastore.get(ws, "fasteners_1")
dataset_images = Dataset.File.from_files(path=(datastore, 'images/'))
mounted_context_images = dataset_images.mount()
mounted_context_images.start()
# Get the mount path
images_dir = mounted_context_images.mount_point
images_dir_folders = [folder for folder in os.listdir(images_dir) if folder not in exclude_folders]
#dataset_labels
dataset_labels = Dataset.File.from_files(path=(datastore, 'labels/'))
mounted_context_labels = dataset_labels.mount()
mounted_context_labels.start()
# Get the mount path
labels_dir = mounted_context_labels.mount_point
labels_dir_folders = [folder for folder in os.listdir(labels_dir) if folder not in exclude_folders]
classes_file = labels_dir_folders

Available datastores: ['azureml_globaldatasets', 'fasteners_1', 'workspacefilestore', 'workspaceworkingdirectory', 'workspaceartifactstore', 'workspaceblobstore']
Images Dataset mounted at: /tmp/tmpy9owx32p ['Cable Straps Edge Mount', 'Cable Straps Hole Mount', 'Tape-on Locators Edge Mount', 'Tape-on Locators Hole Mount']
Labels Dataset mounted at: /tmp/tmphayze_bw ['Cable Straps Edge Mount', 'Cable Straps Hole Mount', 'Tape-on Locators Edge Mount', 'Tape-on Locators Hole Mount']
classes_file: ['Cable Straps Edge Mount', 'Cable Straps Hole Mount', 'Tape-on Locators Edge Mount', 'Tape-on Locators Hole Mount']


In [18]:
# Get the list of class names (subdirectories in images/)
class_names = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
num_classes = len(class_names)+ 1
# Lists to store file paths for each split
train_files = []
test_files = []
# Split the data and store file paths
for class_name in class_names:
    image_class_dir = os.path.join(images_dir, class_name)
    label_class_dir = os.path.join(labels_dir, class_name)
    image_files = [f for f in os.listdir(image_class_dir)]
    random.shuffle(image_files)

    num_images = len(image_files)
    train_split = int(train_ratio * num_images)
    test_split = int(test_ratio * num_images)

    train_image_files = image_files[:train_split]
    test_image_files = image_files[train_split:train_split + test_split]

    for split, files in zip(['train', 'test'], [train_image_files, test_image_files]):
        for file_name in files:
            image_path = os.path.join(image_class_dir, file_name)
            label_path = os.path.join(label_class_dir, file_name.replace(os.path.splitext(file_name)[1], '.txt'))
            if split == 'train':
                train_files.append((image_path, label_path, class_name))
            elif split == 'test':
                test_files.append((image_path, label_path, class_name))

len tarain: 275
len tarain: 77


In [19]:
class CustomDataset(Dataset):
    def __init__(self, file_list, classes_file, image_size=(224, 224), augmentation=False):
        self.file_list = file_list
        self.classes = list_subfolders(classes_file)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.new_width, self.new_height = image_size
        self.resize = Resize(height=self.new_height, width=self.new_width)
        self.normalize = NormalizeTransform(mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225])
        self.augmentations = get_augmentations() if augmentation else []

    def __len__(self):
        return len(self.file_list) * (len(self.augmentations) + 1)

    def __getitem__(self, idx):
        img_idx = idx // (len(self.augmentations) + 1)
        aug_idx = idx % (len(self.augmentations) + 1)
        img_path, label_path, _ = self.file_list[img_idx]

        image = Image.open(img_path).convert("RGB")
        img = np.array(image)
        img = self.resize(image=img)['image']

        bboxes, labels = [], []
        with open(label_path, "r") as f:
            for line in f:
                if line.startswith("Class:"):
                    class_name = line.split(":")[1].strip().strip('"')
                elif line.startswith("Bounding Box:"):
                    bbox = eval(line.split(":")[1].strip())
                    bboxes.append(bbox)
                    labels.append(self.classes.index(class_name) + 1)

        if not bboxes:
            return self.__getitem__((idx + 1) % len(self))

        # Scale bboxes
        orig_width, orig_height = image.size
        scale_x, scale_y = self.new_width / orig_width, self.new_height / orig_height
        bboxes = [[x1 * scale_x, y1 * scale_y, x2 * scale_x, y2 * scale_y] for x1, y1, x2, y2 in bboxes]

        if aug_idx != 0:
            aug = self.augmentations[aug_idx - 1]
            augmented = aug(image=img, bboxes=bboxes, class_labels=labels)
            img = augmented['image']
            bboxes = augmented['bboxes']
            labels = augmented['class_labels']
            img = torch.from_numpy(img.astype(np.float32) / 255.0).permute(2, 0, 1)
        else:
            img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0

        img = self.normalize(img)
        return idx, img, {"boxes": torch.tensor(bboxes, dtype=torch.float32),
                          "labels": torch.tensor(labels, dtype=torch.int64)}
def list_subfolders(main_folder):
    if isinstance(main_folder, list):
        return main_folder
    if not os.path.isdir(main_folder):
        raise ValueError(f"The provided path '{main_folder}' is not valid.")
    return [name for name in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, name))]

def custom_collate(batch):
    images, targets = zip(*batch)
    return torch.stack(images, 0), list(targets)

def get_augmentations():
    bbox_params = BboxParams(format="pascal_voc", label_fields=["class_labels"])
    return [
        Compose([HorizontalFlip()], bbox_params=bbox_params),
        Compose([VerticalFlip()], bbox_params=bbox_params)
    ]

class NormalizeTransform:
    def __init__(self, mean, std):
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def __call__(self, tensor):
        return (tensor - self.mean) / self.std

def denormalize(tensor, mean, std):
    mean = torch.tensor(mean).view(-1, 1, 1)
    std = torch.tensor(std).view(-1, 1, 1)
    return tensor * std + mean

transform = ToTensorV2()

def save_model(model, directory, model_name="custom_object_detection_model.pth"):
    os.makedirs(directory, exist_ok=True)
    save_path = os.path.join(directory, model_name)
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

def load_model(model, directory, model_name):
    load_path = os.path.join(directory, model_name)
    model.load_state_dict(torch.load(load_path))
    print(f"Model loaded from {load_path}")
    return model

/anaconda/envs/Object_Detection/lib/python3.10/site-packages/albumentations/core/composition.py:250: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [20]:
# Create Dataset instances
train_dataset = CustomDataset(file_list = train_files, classes_file=  classes_file , augmentation= False, image_size=(224,224))
test_dataset = CustomDataset(file_list= test_files, classes_file=  classes_file)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers= num_workers,collate_fn=lambda batch: tuple(zip(*batch)))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda batch: tuple(zip(*batch)))

Number of training images: 275
Number of validation images: 77


In [21]:
# Specify the backbone
backbone_1 = resnet_fpn_backbone("resnet50", pretrained= False) # or pretrained=True if you want a pretrained backbone
backbone_2 = resnet_fpn_backbone("resnet101", pretrained= False) # or pretrained=True if you want a pretrained backbone
backbone_3 = resnet_fpn_backbone("resnet34", pretrained= False) # or pretrained=True if you want a pretrained backbone

backbone = [backbone_1, backbone_2, backbone_3]

# Define custom anchor sizes and aspect ratios
anchor_sizes = ((32,), (64,), (128,),(140,),(224,)) # Sizes for each FPN level
aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)  # Aspect ratios for all levels
# Create a custom AnchorGenerator
custom_anchor_generator = AnchorGenerator(sizes=anchor_sizes, aspect_ratios=aspect_ratios)

/anaconda/envs/Object_Detection/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/anaconda/envs/Object_Detection/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/Object_Detection/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/anaconda/envs/Object_Detection/lib/python3.10/site-packages/

In [22]:
model_path_list = ["1000204_.pth", "1000206_.pth", "1000202_.pth", "1000203_.pth", "1000201_.pth", "1000205_.pth", "1000207_.pth", "1000208_.pth"]

# Load the pretrained model
def making_models_list(backbone_list, model_path_list, num_classes, pred_per_image, device, custom_anchor_generator):
    model_list = []
    backbones_list = []

    for path in model_path_list:   
        if path in ["1000203_.pth"]: #ResNet101
            backbone = backbone_list[1]
            backbones_list.append("ResNet101")
        elif path in ["1000202_.pth"]:  #ResNet34
            backbone = backbone_list[2]
            backbones_list.append("ResNet34")
        else:                            #ResNet50
            backbone = backbone_list[0]
            backbones_list.append("ResNet50")
        checkpoint = torch.load(path, map_location = device)
        model = torchvision.models.detection.FasterRCNN(backbone,
                                                        num_classes= num_classes,
                                                        #class_weights=class_weights,
                                                        rpn_pre_nms_top_n_train=100,
                                                        rpn_post_nms_top_n_train=100,
                                                        rpn_nms_thresh= 0.5, #Lower (e.g., 0.3): Stricter filtering (removes more overlapping boxes),
                                                        #Higher (e.g., 0.6): Allows more overlapping boxes (useful if objects are close together).
                                                        rpn_score_thresh=0.4,
                                                        rpn_anchor_generator= custom_anchor_generator,
                                                        box_fg_iou_thresh= 0.5, # Positive threshold,
                                                        box_bg_iou_thresh= 0.5) # Negative threshold)

        representation_size = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(representation_size, num_classes)
        model.roi_heads.detections_per_img = pred_per_image  # Reduce number of false positives
        #print(model.roi_heads)
        #print(model.roi_heads.__dict__.keys())
        model.load_state_dict(checkpoint["model_state_dict"], strict=False)
        model_list.append(model)
    return model_list, backbone_list

In [23]:
models, backbones = making_models_list(backbone_list=backbone, model_path_list= model_path_list, num_classes= num_classes,
                                                pred_per_image =pred_per_image, device=device,
                                                custom_anchor_generator = custom_anchor_generator)

In [24]:
import math
from torch.utils.data import WeightedRandomSampler
from torchvision.ops import box_iou

weights = {str(i): 1/len(train_dataset) for i in range(len(train_dataset))}
# Create a sampler based on the weights

def compute_weighted_error(model, sample_weights, dataset, device):
    scaler = torch.cuda.amp.GradScaler()
    weights_list = [sample_weights[str(i)] for i in range(len(dataset))]

    sampler = WeightedRandomSampler(weights_list, num_samples=len(weights), replacement=True)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers= num_workers,
                                                sampler= sampler, collate_fn=lambda batch: tuple(zip(*batch)))
    model.to(device)
    model.train()
    total_loss = 0
    update_status= {}
    for idxs, images, targets in dataloader:
        idx = [i for i in idxs]
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
            for k in range(len(images)):
                loss_dict = model(images[k].unsqueeze(0), [targets[k]])
                loss = sum(loss for loss in loss_dict.values())

                if 'loss_classifier' in loss_dict and 'loss_box_reg' in loss_dict:
                    if loss_dict['loss_classifier'] < 0.05 and loss_dict['loss_box_reg'] < 0.05:
                        update_status[str(idx[k])] = 'NO'
                        print(f" id : {str(idx[k])} value: {update_status[str(idx[k])]}")
                    else:
                        update_status[str(idx[k])] = 'YES'
                        total_loss += loss * weights[str(idx[k])]
                        print(f" id : {str(idx[k])} value: {update_status[str(idx[k])]}")
                elif isinstance(loss_dict, list):
                    total_classifier_loss = sum(d.get('loss_classifier', 0) for d in loss_dict)
                    total_box_reg_loss = sum(d.get('loss_box_reg', 0) for d in loss_dict)
                    if total_classifier_loss <= 0.05 and total_box_reg_loss < 0.05:
                            update_status[str(idx[k])] = 'NO'
                            print(f" id : {str(idx[k])} value: {update_status[str(idx[k])]}")
                    else:
                            update_status[str(idx[k])] = 'YES'
                            print(f" id : {str(idx[k])} value: {update_status[str(idx[k])]}")
                            total_loss += loss * weights[str(idx[k])]

    error = total_loss / len(dataloader.dataset)
    print("Model training Done!")
    return error, update_status

def adaboost_faster_rcnn(models, dataset, sample_weights, device):
    model_weights = []

    for i, model in enumerate(models):
        print("Start updating weights!")
        error, update_status = compute_weighted_error(model=model, sample_weights = sample_weights,
                                                                                dataset=dataset,  device=device)
        if error == 0:
            alpha = 1  # Strong classifier
        else:
            alpha = 0.5 * math.log((1 - error) / error)

        model_weights.append(alpha)

        for i in sample_weights.keys():
            if i in update_status.keys():
                if update_status[i] == "NO":  # Correctly referencing dictionary keys
                    sample_weights[i] *= math.exp(-alpha)
                else:
                    sample_weights[i] *= math.exp(alpha)

        # Normalize sample weights
        total_weight = np.sum(list(sample_weights.values()))  # Convert to list for sum
        if total_weight > 0:
            sample_weights = {k: v / total_weight for k, v in sample_weights.items()}

    return model_weights, sample_weights

def weighted_nms(predictions, model_weights, iou_threshold=0.5):
    batch_size = len(predictions[0])  # Get batch size
    final_preds = []

    for batch_idx in range(batch_size):
        combined_preds = []

        for preds, weight in zip(predictions, model_weights):
            for box, score, label in zip(
                preds[batch_idx]['boxes'], preds[batch_idx]['scores'], preds[batch_idx]['labels']
            ):
                combined_preds.append((box, score * weight, label))

        combined_preds.sort(key=lambda x: x[1], reverse=True)

        filtered_preds = []
        while combined_preds:
            best = combined_preds.pop(0)
            filtered_preds.append(best)

            combined_preds = [p for p in combined_preds if iou(p[0], best[0]) < iou_threshold]

        # Convert back to the required format
        if filtered_preds:
            final_preds.append({
                "boxes": torch.stack([p[0] for p in filtered_preds]),
                "scores": torch.tensor([p[1] for p in filtered_preds]),
                "labels": torch.tensor([p[2] for p in filtered_preds])
            })
        else:
            # Handle empty case
            final_preds.append({"boxes": torch.tensor([]), "scores": torch.tensor([]), "labels": torch.tensor([])})

    return final_preds

def iou(box1, box2):
    box1_tensor = torch.tensor(box1).unsqueeze(0)
    box2_tensor = torch.tensor(box2).unsqueeze(0)
    return box_iou(box1_tensor, box2_tensor).item() # Returns a single IoU value

In [25]:
# Apply AdaBoost
model_weights , sample_weights = adaboost_faster_rcnn(models= models, dataset = train_dataset,
                                                                sample_weights =weights, device=device)

/tmp/ipykernel_3146/2660485587.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


 id : 20 value: YES
 id : 183 value: YES
 id : 55 value: YES
 id : 39 value: YES
 id : 165 value: YES
 id : 97 value: YES
 id : 21 value: YES
 id : 178 value: YES
 id : 222 value: YES
 id : 116 value: YES
 id : 240 value: YES
 id : 261 value: YES
 id : 192 value: YES
 id : 203 value: YES
 id : 257 value: YES
 id : 217 value: YES
 id : 94 value: YES
 id : 45 value: YES
 id : 98 value: YES
 id : 6 value: YES
 id : 64 value: YES
 id : 27 value: YES
 id : 199 value: YES
 id : 28 value: YES
 id : 230 value: YES
 id : 188 value: YES
 id : 218 value: YES
 id : 168 value: YES
 id : 152 value: YES
 id : 61 value: YES
 id : 3 value: YES
 id : 184 value: YES
 id : 220 value: YES
 id : 105 value: YES
 id : 102 value: YES
 id : 31 value: YES
 id : 169 value: YES
 id : 2 value: YES
 id : 172 value: YES
 id : 19 value: YES
 id : 272 value: YES
 id : 86 value: YES
 id : 187 value: YES
 id : 68 value: YES
 id : 208 value: YES
 id : 80 value: YES
 id : 139 value: YES
 id : 30 value: YES
 id : 192 value:

In [26]:
print(model_weights)

[2.5798268603328642, 1.355877325060082, 1.7440529047542193, 1.7606545104640545, 1.4459289276104286, 1.3422684219743737, 1.4650970537150028, 1.94854436956064]


In [27]:
metric = MeanAveragePrecision()

#images, _ = next(iter(val_loader))
for model in models:
    model.to(device)
    model.eval()
with torch.no_grad():
    for idxs, images, targets in test_loader:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                """for img in images:
                        preds1 = models[0](img.unsqueeze(0))
                        print(preds1)
                        preds2 = models[1](img.unsqueeze(0))
                        preds3 = models[2](img.unsqueeze(0))
                        preds4 = models[3](img.unsqueeze(0))"""
                preds = []
                for model in models:

                    pred = model(images)
                    #print(preds1)
                    #preds2 = models[1](images)
                    #preds3 = models[2](images)
                    #preds4 = models[3](images)
                    preds.append(pred)
                # Combine predictions
                final_preds = weighted_nms(preds, model_weights)

                gt_targets = [
                            dict(
                                boxes=t["boxes"].detach().cpu(),
                                labels=t["labels"].detach().cpu()
                            ) for t in targets
                        ]

                metric.update(final_preds, gt_targets)

                #print(final_preds)

    mAP_result = metric.compute()
    mAP_score = mAP_result["map"].item()
    mAP_50 = mAP_result["map_50"].item()
    print(mAP_score)
    print(mAP_50)

/tmp/ipykernel_3146/2660485587.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box1_tensor = torch.tensor(box1).unsqueeze(0)
/tmp/ipykernel_3146/2660485587.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box2_tensor = torch.tensor(box2).unsqueeze(0)


0.7960504293441772
0.962081253528595


In [29]:
# Save each model with its assigned AdaBoost weight
model_data = {
    "models": [],
    "model_weights": model_weights,  # Save AdaBoost model weights (alphas)
    "model_backbone": backbones,  #list of backbones of the models
}

for i, model in enumerate(models):
    model_state = {
        "model_state_dict": model.state_dict(),
        "alpha": model_weights[i],  # Save the weight assigned by AdaBoost
    }
    model_data["models"].append(model_state)

torch.save(model_data, "adaboost_faster_rcnn.pth")
print("Model and weights saved successfully!")

Model and weights saved successfully!


In [30]:
import torch

torch.cuda.empty_cache()
class_names = test_dataset.classes

def test_model_adaboost(models, test_loader, device, class_names, model_weights, visualize=False):
    # Move models to device and set to eval mode
    for model in models:
        model.to(device)
        model.eval()

    # Initialize metrics and accumulators
    metric = MeanAveragePrecision()
    total_test_loss = 0.0

    with torch.no_grad():
        for idxs, images, targets in test_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Collect predictions from all models
            all_predictions = []
            for model in models:
                pred = model(images)
                all_predictions.append(pred)

            # Apply Weighted NMS using AdaBoost weights
            final_preds = weighted_nms(all_predictions, model_weights)

            # Compute loss (optional, only for evaluation)
            total_loss = 0
            for model in models:
                loss_dict = model(images, targets)
                if isinstance(loss_dict, list):
                    losses = sum(sum(loss_value.sum() for loss_value in loss.values()) for loss in loss_dict)
                elif isinstance(loss_dict, dict):
                    losses = sum(loss_value.sum() for loss_value in loss_dict.values())
                else:
                    raise TypeError(f"Unexpected loss format: {type(loss_dict)}")
                total_loss += losses.item()

            total_test_loss += total_loss / len(models)  # Averaging loss over models

            # Convert ground truth to required format for mAP computation
            gt_targets = [
                dict(
                    boxes=t["boxes"].detach().cpu(),
                    labels=t["labels"].detach().cpu()
                ) for t in targets
            ]

            # Update metric
            metric.update(final_preds, gt_targets)

            # Optional: Visualize some predictions
            if visualize:
                visualize_predictions(images, final_preds, class_names)

    # Compute Mean Average Precision (mAP)
    mAP_result = metric.compute()
    mAP_score = mAP_result["map"].item()
    print(f"\nMean Average Precision (mAP): {mAP_score:.4f}")

    # Compute average test loss
    avg_test_loss = total_test_loss / len(test_loader)
    print(f"\nAverage Test Loss: {avg_test_loss:.4f}")

    return mAP_score, avg_test_loss

def visualize_predictions(images, predictions, class_names):

    for i, img in enumerate(images):
        # Denormalize image
        img = denormalize(img.cpu(), mean, std)
        img = F_transform.to_pil_image(img)  # Convert to PIL image

        # Create figure
        fig, ax = plt.subplots(1, figsize=(8, 8))
        ax.imshow(img)

        # Get predicted boxes, labels, and scores
        pred_boxes = predictions[i]['boxes'].cpu().numpy()
        pred_labels = predictions[i]['labels'].cpu().numpy()
        pred_scores = predictions[i]['scores'].cpu().numpy()

        if len(pred_scores) == 0:
            print("No predictions found for this image.")
            continue

        # Find the highest confidence score
        pred_scores_tensor = torch.tensor(pred_scores) 
        max_score_idx = pred_scores_tensor.argmax()  # Get index of highest score

        # Get highest confidence box, label, and score
        best_box = pred_boxes[max_score_idx]
        best_label = pred_labels[max_score_idx]
        best_score = pred_scores[max_score_idx]
        ixd_best_label = pred_labels[max_score_idx].item()
        predicted_label_name = class_names[ixd_best_label - 1]

        # Draw Highest Confidence Bounding Box (Red)
        xmin, ymin, xmax, ymax = best_box
        rect = plt.Rectangle(
            (xmin, ymin), xmax - xmin, ymax - ymin,
            fill=False, color="red", linewidth=2
        )
        ax.add_patch(rect)

        # Display label and score
        ax.text(
            #xmin, ymin - 5, f"{predicted_label_name}: {best_score:.2f}",
            xmin, ymin - 5, f"{predicted_label_name}",
            color="red", fontsize=12, bbox=dict(facecolor='white', alpha=0.7)
        )

        plt.axis("off")
        plt.title("Highest Confidence Prediction")
        plt.show()

In [ ]:
# Run the model on the test dataset
# Run evaluation using AdaBoost predictions
mAP_score, avg_loss = test_model_adaboost(
    models=models,
    test_loader=test_loader,
    device=device,
    class_names=test_dataset.classes,
    model_weights=model_weights,
    visualize=True  # Set to False if you don't need visualization
)
print(mAP_score,avg_loss)

/tmp/ipykernel_3146/2660485587.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box1_tensor = torch.tensor(box1).unsqueeze(0)
/tmp/ipykernel_3146/2660485587.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box2_tensor = torch.tensor(box2).unsqueeze(0)
